<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

In [ ]:
!unzip mallet-2.0.8.zip

In [1]:

from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python -m pip install --upgrade pip
!pip install pyLDAvis
# import pyLDAvis
import pyLDAvis.gensim_models # don't skip this

from pprint import pprint
import time
import pandas as pd
import re
import string
import nltk
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections 
try:
    from collections.abc import Callable  # noqa
except ImportError:
    from collections import Callable  # noqa



In [3]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [71]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

bug_report = pd.read_csv(csv_file)
bug_report

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...,CLOSED,FIXED,2,10/10/2001 21:34,2/9/2012 15:57
1,2,P5,Team,NaN,Opening repository resources doesnt honor type (1GE6I1Q),Opening repository resource always open the default text editor and doesnt; \thonor any mapping between resource types and editors.; \tAs a result it is not possible to view the contents of an image (*.gif file) in a sensible way.; ; NOTES:; EG (5/24/2001 5:13:45 AM); \tAgreed that this is needed eventually but not critical for June. \t; \tThere should also be a Java outliner when editing a java resource from the repository.; \tMoving to VCM.; ; KM (5/24/01 2:09:21 PM); \tIn general you cannot open a specialized editor on a remote resource because they can sometimes; \tbe dependent on other resources and expect these to be local. For example; an HTML page which; \treferences a graphic; the HTML editor is going to try to look for the graphic to display it.; ; \tYou cant be sure that the editor was written to accomodate remote or missing resources.; \tWe discussed this some time ago with Dave T. and felt it was safer to just use vanilla editors for now.; ; KM (5/24/01 4:42:09 PM); \...,RESOLVED,FIXED,2,10/10/2001 21:34,5/7/2002 10:33
2,3,P5,Team,NaN,Sync does not indicate deletion (1GIEN83),KM (10/2/2001 5:55:18 PM); \tThis PR about the deletion indicator in the sync viewer being too subtle.; \tThis would be even more true for someone with vision problems.; \tNE has good suggestion of using the label of the compare text viewer.; \tWe may also consider a better icon.; \tOn to the PR....; ; - delete file from project; - synchronize; - there is no indication that the file has been deleted from the workspace; - it looks the same as if the file is in the workspace but is empty; ; NOTES:; ; KM (8/15/2001 3:37:53 PM); \tIt should appear as a rightfacing - arrow in the sync view.; \tSent screen cap to NE requesting clarification.; ; NE (8/20/01 9:13:38 AM); \tThe - in the arrow is clearer when there are other arrows present too (the + and change ones).; \tOtherwise; its easy to miss the distinction; especially when the users attention is focussed; \ton the text compare panes.; \tId expect the Workspace file: label to indicate that it has been deleted.; \tThe Repository file: ...,RESOLVED,FIXED,2,10/10/2001 21:34,5/7/2010 10:28
3,4,P5,Team,NaN,need better error message if catching up over read-only resource (1GF69TF),- become synchronized with some project in a repository; - use a different Eclipse to make a change to a file resource within this project and release it to the repository; - in the original Eclipse mark this file resource as being read-only (select it; right-click -> Properties; change; OK); - select the file resource; right-click -> Team -> Synchronize with Stream; - in the subsequent comparison view select the file resource; right-click -> Catchu

In [ ]:
%%time

bug_report = bug_report.drop(columns=['Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
bug_report = bug_report.dropna(axis=0, subset=['Description'])
bug_report[bug_report['Description'].map(len) > 4]

# bug_report.head(1)

In [9]:
%%time

bug_report["Description"] = bug_report["Description"].str.replace("Fixed in HEAD", "")
bug_report["Description"] = bug_report["Description"].str.replace("Has been marked as read-only", " ")

CPU times: user 489 ms, sys: 0 ns, total: 489 ms
Wall time: 493 ms


In [10]:
%%time

def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

bug_report['Description'] = bug_report['Description'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))
bug_report['Title'] = bug_report['Title'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))

# bug_report.loc[[0]]

CPU times: user 15.7 s, sys: 22.4 ms, total: 15.8 s
Wall time: 16.7 s


In [73]:
result = bug_report.copy(deep=True)
result.reset_index(drop=True, inplace=True)
result.to_csv('drive/My Drive/result_reports.csv')
result

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...,CLOSED,FIXED,2,10/10/2001 21:34,2/9/2012 15:57
1,2,P5,Team,NaN,Opening repository resources doesnt honor type (1GE6I1Q),Opening repository resource always open the default text editor and doesnt; \thonor any mapping between resource types and editors.; \tAs a result it is not possible to view the contents of an image (*.gif file) in a sensible way.; ; NOTES:; EG (5/24/2001 5:13:45 AM); \tAgreed that this is needed eventually but not critical for June. \t; \tThere should also be a Java outliner when editing a java resource from the repository.; \tMoving to VCM.; ; KM (5/24/01 2:09:21 PM); \tIn general you cannot open a specialized editor on a remote resource because they can sometimes; \tbe dependent on other resources and expect these to be local. For example; an HTML page which; \treferences a graphic; the HTML editor is going to try to look for the graphic to display it.; ; \tYou cant be sure that the editor was written to accomodate remote or missing resources.; \tWe discussed this some time ago with Dave T. and felt it was safer to just use vanilla editors for now.; ; KM (5/24/01 4:42:09 PM); \...,RESOLVED,FIXED,2,10/10/2001 21:34,5/7/2002 10:33
2,3,P5,Team,NaN,Sync does not indicate deletion (1GIEN83),KM (10/2/2001 5:55:18 PM); \tThis PR about the deletion indicator in the sync viewer being too subtle.; \tThis would be even more true for someone with vision problems.; \tNE has good suggestion of using the label of the compare text viewer.; \tWe may also consider a better icon.; \tOn to the PR....; ; - delete file from project; - synchronize; - there is no indication that the file has been deleted from the workspace; - it looks the same as if the file is in the workspace but is empty; ; NOTES:; ; KM (8/15/2001 3:37:53 PM); \tIt should appear as a rightfacing - arrow in the sync view.; \tSent screen cap to NE requesting clarification.; ; NE (8/20/01 9:13:38 AM); \tThe - in the arrow is clearer when there are other arrows present too (the + and change ones).; \tOtherwise; its easy to miss the distinction; especially when the users attention is focussed; \ton the text compare panes.; \tId expect the Workspace file: label to indicate that it has been deleted.; \tThe Repository file: ...,RESOLVED,FIXED,2,10/10/2001 21:34,5/7/2010 10:28
3,4,P5,Team,NaN,need better error message if catching up over read-only resource (1GF69TF),- become synchronized with some project in a repository; - use a different Eclipse to make a change to a file resource within this project and release it to the repository; - in the original Eclipse mark this file resource as being read-only (select it; right-click -> Properties; change; OK); - select the file resource; right-click -> Team -> Synchronize with Stream; - in the subsequent comparison view select the file resource; right-click -> Catchu

In [12]:
%%time

def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

bug_report['Title'] = bug_report['Title'].apply(lambda x: wordTokenizationAndLemmatization(x))
bug_report['Description'] = bug_report['Description'].apply(lambda x: wordTokenizationAndLemmatization(x))

# bug_report.head()

CPU times: user 30.4 s, sys: 308 ms, total: 30.7 s
Wall time: 33.3 s


In [13]:
%%time

duplicated_reports = bug_report.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.to_csv('duplicate_reports.csv')
# duplicated_reports

CPU times: user 274 ms, sys: 10.8 ms, total: 284 ms
Wall time: 286 ms


In [14]:
%%time

master_reports = bug_report[bug_report.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.to_csv('master_reports.csv')
# master_reports

CPU times: user 1.06 s, sys: 42.9 ms, total: 1.1 s
Wall time: 1.11 s


In [15]:
%%time

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for key, value in dictionary.iteritems():
#   print(key, value)
  count += 1
  if count > 100:
      break


CPU times: user 1.99 s, sys: 10.1 ms, total: 2 s
Wall time: 2.02 s


In [16]:
%%time

bag_of_words_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in master_reports['Description']]
# bag_of_words_corpus[0]

CPU times: user 2.01 s, sys: 76.9 ms, total: 2.08 s
Wall time: 2.1 s


In [ ]:
%%time

bag_of_words_doc = bag_of_words_corpus[0]

for i in range(len(bag_of_words_doc)):
    print("{}. \"{}\" word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

In [ ]:
mallet_path = '/content/mallet-2.0.8/bin/mallet'

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=bag_of_words_corpus, num_topics=10, id2word=dictionary)

In [ ]:
from pprint import pprint
# display topics
pprint(ldamallet.show_topics(formatted=False))

In [ ]:
import pickle
pickle.dump(ldamallet, open("drive/My Drive/ldamallet.pkl", "wb"))

In [17]:
ldamallet = pickle.load(open("drive/My Drive/ldamallet.pkl", "rb"))
# ldamallet.print_topics()

In [ ]:
%%time


for c in range(10):
    exec('topic{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=ldamallet[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic{} = topic{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic{} = topic{}.reset_index(drop=True)'.format(c,c))
            exec('topic{}.to_csv("topic{}.csv")'.format(c,c))

CPU times: user 1h 32min 59s, sys: 3min 53s, total: 1h 36min 52s
Wall time: 1h 39min 56s


In [18]:
%%time

for c in range(10):
  exec('topic{} = pd.read_csv("topic{}.csv", error_bad_lines=False, engine="python")'.format(c,c))
  exec("topic{}= topic{}.drop(columns=['Unnamed: 0'])".format(c,c))
  exec("topic{}['Description'] = topic{}['Description'].map(wordTokenizationAndLemmatization)".format(c,c))

CPU times: user 10.6 s, sys: 152 ms, total: 10.8 s
Wall time: 11 s


In [19]:
%%time

for i in range(10):
  exec('sent{} = []'.format(i))
  exec('x = topic{}'.format(i))
  for j in range(len(x)):
    exec('sent{}.append(topic{}.Description[{}])'.format(i,i,j))

CPU times: user 1.56 s, sys: 6.6 ms, total: 1.57 s
Wall time: 1.57 s


In [ ]:
%%time

#Length of all the cluster
for sent in range(10):
  exec('print(len(sent{}))'.format(sent))
  

In [ ]:
%%time

corpus = bag_of_words_corpus

lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

pprint(lda_model.print_topics())

In [24]:
file = open('drive/My Drive/dictionary.pickle', 'wb')

# dump information to that file
pickle.dump(dictionary, file)

In [27]:
f= open('drive/My Drive/bow_corpus.pickle', 'wb')

# dump information to that file
pickle.dump(bag_of_words_corpus, f)

In [ ]:
f = open('drive/My Drive/bow_corpus.pickle', 'rb')
bow_corpus=pickle.load(f)

file = open('drive/My Drive/dictionary.pickle', 'rb')
dictionary=pickle.load(file)



In [ ]:
# later on, load trained model from file
lda_model =  models.LdaModel.load('drive/My Drive/lda_model.model')
pprint(lda_model.print_topics())

In [ ]:
lda_model.save('drive/My Drive/lda_model.model')

In [ ]:
%%time

print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=bug_report['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
%%time

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

In [68]:
unseen_document = "setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with."

topic=wordTokenizationAndLemmatization(unseen_document)

# bow_vector = topic=ldamallet[dictionary.doc2bow(wordTokenizationAndLemmatization(unseen_document))]
print(topic)

# for index, score in sorted(ldamallet[bow_vector], key=lambda tup: -1*tup[1]):
#     print("Score: {}\t Topic: {}".format(score, ldamallet.print_topic(index, 10)))

['project', 'contain', 'resource', 'release', 'project', 'resource', 'external', 'editor', 'paintshop', 'external', 'editor', 'navigator', 'resource', 'verify', 'change', 'release', 'project', 'release', 'navigator', 'resource', 'verify', 'change', 'problem', 'refresh', 'workspace', 'change', 'release', 'open', 'resource']


In [ ]:
%%time


for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))
    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(c,c))


In [ ]:
%%time

for c in range(10):
  exec('topic_{} = pd.read_csv("topic_{}.csv", error_bad_lines=False, engine="python")'.format(c,c))
  exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
  exec("topic_{}['Description'] = topic_{}['Description'].map(wordTokenizationAndLemmatization)".format(c,c))

CPU times: user 10.7 s, sys: 295 ms, total: 11 s
Wall time: 11 s


In [ ]:
%%time

for i in range(10):
  exec('sent_{} = []'.format(i))
  exec('x = topic_{}'.format(i))
  for j in range(len(x)):
    exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

CPU times: user 1.72 s, sys: 11.7 ms, total: 1.73 s
Wall time: 1.75 s


In [ ]:
%%time

#Length of all the cluster
for sent in range(10):
  exec('print(len(sent_{}))'.format(sent))
  

In [23]:
!pip install glove-python-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.9/974.9 kB 10.4 MB/s eta 0:00:00


In [24]:
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile
from glove import Glove, Corpus
import glove

In [33]:
%%time

for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent{}'.format(cluster))    # Path to a corpus file
  vector_size = 100                           # Dimensionality of the word vectors
  w = 6                                       #Maximum distance between the current and predicted word within a sentence
  min_count = 5                               # Ignores all words with total frequency lower than this.
  CBoW = 0                                    # Training algorithm: 1 for skip-gram; otherwise CBOW(The model tries to predict the target word by trying to understand the context of the surrounding words.).
  iterations = 100

  exec('w2vmodel_ldamallet{} = Word2Vec(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  exec('path = get_tmpfile("drive/My Drive/word2vec_ldamallet{}.model")'.format(cluster))
  exec('w2vmodel_ldamallet{}.save("drive/My Drive/word2vec_ldamallet{}.model")'.format(cluster, cluster))

CPU times: user 4min 49s, sys: 2.55 s, total: 4min 51s
Wall time: 2min 46s


In [34]:
%%time

for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent{}'.format(cluster))
  vector_size = 100
  w = 6
  min_count = 5
  CBoW = 0
  iterations = 100

  #Training FastText model for each cluster
  exec ('ftmodel_ldamallet{} = FastText(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("drive/My Drive/ftmodel_ldamallet{}.model")'.format(cluster))
  exec('ftmodel_ldamallet{}.save("drive/My Drive/ftmodel_ldamallet{}.model")'.format(cluster, cluster))

CPU times: user 31min 57s, sys: 7.77 s, total: 32min 5s
Wall time: 17min 17s


In [ ]:
%%time

for cluster in range(10):
  vector_size = 100
  exec('glove_corpus{}=Corpus()'.format(cluster, cluster)) 
  exec('glove_corpus{}.fit(sent{})'.format(cluster, cluster))
  exec('glove_ldamallet{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
  exec('glove_ldamallet{}.fit(glove_corpus{}.matrix, epochs=1000, no_threads=3, verbose=True)'.format(cluster, cluster))
  exec('transformer = lambda dictionary2:glove_m{}.transform_paragraph(words, epochs=1000,ignore_missing=False)'.format(cluster, cluster))
  exec('glove_ldamallet{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("drive/My Drive/glove_ldamallet{}.model")'.format(cluster))
  exec('glove_ldamallet{}.save("drive/My Drive/glove_ldamallet{}.model")'.format(cluster, cluster))

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from scipy.spatial import distance
warnings.filterwarnings("ignore", category=DeprecationWarning)
from glove import Glove, Corpus

In [26]:
test = duplicated_reports
test.head()
print(len(test))

14398


In [27]:
for mod in range(10):
  #import all the trained Word2Vec models
  exec('w2vmodel_ldamallet{} = Word2Vec.load("drive/My Drive/word2vec_ldamallet{}.model")'.format(mod, mod))

  #import all the trained FastText models
  exec('ftmodel_ldamallet{} = FastText.load("drive/My Drive/ftmodel_ldamallet{}.model")'.format(mod, mod))

  #import all the trained GloVe models
  exec('glove_ldamallet{} = Glove.load("drive/My Drive/glove_ldamallet{}.model")'.format(mod, mod))

In [ ]:
for mod in range(10):
    exec("print(list(w2vmodel_ldamallet{}.wv.vocab))".format(mod))

In [28]:
def sim_with_clusters_lda_topn(DR, n):
    vec_bow = dictionary.doc2bow(DR)
    x= ldamallet[vec_bow]
    topic = np.asarray(x)
    # max_sim = int(topic[np.argmax(topic[:,1]),0]) 
    # max_sim
    sim=[]
    x= topic[np.argsort(topic[:,1])[-n:][::-1],0]
    for i in range(len(x)):
        sim.append(int(x[i]))
    # return max_sim
    return sim


In [29]:
def average_word_vectors_glove(words, model, vocabulary, num_features):  
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.  

    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.word_vectors[model.dictionary[word]])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    


def averaged_word_vectorizer_glove(corpus, model, num_features):
    vocabulary = set(model.dictionary)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_glove(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_glove(corpus, model, vocabulary, num_features)
      return np.array(features)


In [30]:
def average_word_vectors_ft(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector
    
def averaged_word_vectorizer_ft(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_ft(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_ft(corpus, model, vocabulary, num_features)
      return np.array(features)

In [31]:
def average_word_vectors_word2vec(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

def averaged_word_vectorizer_word2vec(corpus, model, num_features):
    vocabulary = set(model.wv.vocab)
    if(any(isinstance(i, list) for i in corpus)):
      features = [average_word_vectors_word2vec(tokenized_sentence, model, vocabulary, num_features)
                      for tokenized_sentence in corpus]
      return np.array(features)
    else:
      features = average_word_vectors_word2vec(corpus, model, vocabulary, num_features)
      return np.array(features)

In [32]:
def sim(vec1, vec2): 
  sim1 = 1/(1+distance.euclidean(vec1, vec2))
  sim2 = cosine_similarity(vec1, vec2)
  sim=(sim1+sim2)/2 
  return sim

In [33]:
def fusion3(vec1, vec2, vec3, vec4, vec5, vec6, fusion_no):
    if (fusion_no == '4'):
        master = np.concatenate((vec1, vec2, vec3), axis=1)
        pca = PCA(n_components=100)
        # avg_fit = pca.fit(master)
        master = pca.fit_transform(master)
        vec_duplicate = np.concatenate((vec4, vec5, vec6), axis=0)
        vec_duplicate=[vec_duplicate]
        vec_duplicate = pca.transform(vec_duplicate)
        return vec_duplicate, master

    else:
        raise ValueError('Invalid value for fusion')

In [39]:
def fusion2(vec1, vec2, vec3, vec4, fusion_no):
    if (fusion_no == '2'):
        master = np.concatenate((vec1, vec2), axis=1)
        pca = PCA(n_components=50)
        avg_fit = pca.fit(master)
        master = pca.transform(master)
        vec_duplicate = np.concatenate((vec3, vec4), axis=0)
        vec_duplicate=[vec_duplicate]
        vec_duplicate = pca.transform(vec_duplicate)
        return vec_duplicate, master
    
    else:
        raise ValueError('Invalid value for fusion')

In [55]:
# creation of feature vectors by multimodality feature extraction
def feature_vectors_multi_modality(DR, corpus, model1, model2, model3, fusion_no):

    master_ft1 = averaged_word_vectorizer_ft(corpus=sent, model=model1, num_features=100)
    master_glove2 = averaged_word_vectorizer_glove(corpus=sent, model=model2, num_features=100)
    master_w2v3 = averaged_word_vectorizer_word2vec(corpus=sent, model=model3, num_features=100)

    vec_duplicate1 = averaged_word_vectorizer_ft(corpus=DR, model=model1, num_features=100)
    vec_duplicate2 = averaged_word_vectorizer_glove(corpus=DR, model=model2, num_features=100)
    vec_duplicate3 = averaged_word_vectorizer_word2vec(corpus=DR, model=model3, num_features=100)

    vec_duplicate , master= fusion3(master_ft1,  master_glove2, master_w2v3, vec_duplicate1,  vec_duplicate2, vec_duplicate3, fusion_no)
    # vec_duplicate , master= fusion2(master_ft1, master_w2v3, vec_duplicate1, vec_duplicate3, fusion_no)

    return vec_duplicate,master

In [47]:
def feature_vectors_single_modality(DR, corpus, model1):
  master = averaged_word_vectorizer_ft(corpus=sent, model=model1, num_features=100)

  vec_duplicate = averaged_word_vectorizer_ft(corpus=DR, model=model1, num_features=100)

  vec_duplicate = [vec_duplicate]

  return vec_duplicate, master

In [48]:
def compare_topn(model1, model2, model3, cluster, sent, DR, topn, modal, fusion_no):
    similarity=[]

    if (modal == 'multi'):
    #create feature vectors for duplicate and master reports using multimodality
        vec_duplicate, master= feature_vectors_multi_modality(DR, sent, model1, model2, model3, fusion_no)
    elif (modal == 'single'):
        vec_duplicate, master= feature_vectors_single_modality(DR, sent, model1)
    else:
        raise ValueError('Invalid Modality entered')

    for doc in range(len(master)):
        vec_master = master[doc]
        vec_master= [vec_master]
        unified_sim = sim(vec_duplicate, vec_master)

        similarity.append(unified_sim)

    similarity = np.asarray(similarity)
    similarity= np.concatenate(similarity, axis=0 )
    similarity= np.concatenate(similarity, axis=0 )
    max_similar_reports=similarity.argsort()[-topn:][::-1]

    return(max_similar_reports)

In [57]:
# Recall Rate for Top-2.5K reports (Because Top-N where N = n * topn (2.5K = 3*833)) 
vec_acc=[]
t1 = time.time()
sum = 0
no_of_test_samples= int(200)
for i in range(200):
  sample = test.Description[i] #The test sample (duplicate report)
  n = 1
  max_cluster = sim_with_clusters_lda_topn(sample, n)
  v=[]
  print(i)
  for max in max_cluster:
    exec('cluster = topic{}'.format(max))              #The predicted cluster
    exec('model1 = ftmodel_ldamallet{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
    exec('model2 = glove_ldamallet{}'.format(max))  
    exec('model3 = w2vmodel_ldamallet{}'.format(max))              #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
    exec('sent = topic{}.Description'.format(max))     #The vocabulary for the predicted cluster
    topn = 833                                          #The number of predicted master report for single predicted cluster
    fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
    # modal = 'multi'
    modal = 'multi'                                    #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
     #This will return the Top-N predicted master reports
    max_sim = compare_topn(model1, model2, model3, cluster, sent, sample, topn, modal, fusion_no)
    t2 = time.time()

    #Comparing the predicted value to the ground truth
    for num in max_sim:
      if (cluster.Issue_id[num] == test.Duplicated_issue[i]):
          v.append("1")
      else:
          v.append("0")
  
  if(all(x==v[0] for x in v)):
    vec_acc.append("0")
  else:
    vec_acc.append("1")

print(v)
print(len(vec_acc))
print(vec_acc)
#Evaluating the performance by Recall Rate

for i,num in enumerate(vec_acc):
    sum = sum + int(num)
recall_rate = (sum/len(vec_acc))*100
print("Recall Rate : {} %".format(recall_rate))
print("Time : ", (t2-t1)/60, "min")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',

In [81]:
import random
from tabulate import tabulate

# topic = random.randint(0, 14398)
# print(topic)
# sample = test.Description[topic] #The test sample (duplicate report)
# print(sample)
sample = 'Online Grammar and Writing Checker To Help You Deliver Impeccable, Mistake-free Writing. Grammarly Has a Tool For Just About Every Kind Of Writing You Do. Try It Out For Yourself! Easily Improve Any Text. Find and Add Sources Fast. Eliminate Grammar Errors.'
sample = wordTokenizationAndLemmatization(sample)
print(sample)
max_cluster = sim_with_clusters_lda_topn(sample, 1)
print(max_cluster)

for max in max_cluster:
    exec('cluster = topic{}'.format(max))              #The predicted cluster
    exec('model1 = ftmodel_ldamallet{}'.format(max))              #The trained FastText model for the predicted cluster   (can be changed to other model as well viz. glove or word2vec)
    exec('model2 = glove_ldamallet{}'.format(max))  
    exec('model3 = w2vmodel_ldamallet{}'.format(max))              #The trained Word2Vec model for the predicted cluster   (Doesn't count if using single modality)
    exec('sent = topic{}.Description'.format(max))     #The vocabulary for the predicted cluster
    topn = 10                                          #The number of predicted master report for single predicted cluster
    fusion_no = '4'   #Doesn't count if single modality #The selection of fusion used to fuse the word embeddings of two different models  (4 gives the best results)
    modal = 'multi'
                                      #Whether you want to use single feature extraction model or multi model ( for single, it'll consider just model1)
     #This will return the Top-N predicted duplicate reports
    max_sim = compare_topn(model1, model2, model3, cluster, sent, sample, topn, modal, fusion_no)
    max_sim = max_sim.tolist()
    print(max_sim)
    
    
    for num in max_sim:
      id = cluster.Issue_id[num]

      title = (result.loc[result['Issue_id'] == id, 'Title']).to_string(index = False)
      desc = (result.loc[result['Issue_id'] == id, 'Description']).to_string(index = False)
      priority = (result.loc[result['Issue_id'] == id, 'Priority']).to_string(index = False)
      team = (result.loc[result['Issue_id'] == id, 'Component']).to_string(index = False)

      print('Title: ', title)
      print('Description: ', desc)
      print('Priority:', priority)
      print('Team:', team)
    
   

['online', 'grammar', 'write', 'checker', 'deliver', 'impeccable', 'mistake', 'write', 'grammarly', 'write', 'easily', 'improve', 'source', 'eliminate', 'grammar', 'errors']
[3]
[1936, 11048, 10472, 10598, 11126, 10920, 10055, 5311, 2135, 1795]
Title:  [Tasks] Improvements in sort order
Description:  build I20030128; ; Philippe asks:; ; Is there any chance to get the normal tasks to be sorted behind error markers ; for M5 ?; ; I cant find a PR for this one; though I remember having seen/entered one. The ; current behavior makes it feel like tasks are more severe than errors.; ; ; Nick replies:; ; My feeling is that user generated tasks -are- always more important than ; problems generated by the compiler.; Now that we have auto-generated tasks; they could be treated differently; but ; it would be strange to have tasks show up in two different places with problems ; in between.; ; ; Erich opines:; ; Considering the lifetime of a compiler error and a user task then we see that; compiler 